# 自动求梯度
---
- 在深度学习中，我们经常需要对函数求梯度（gradient）。PyTorch提供的autograd包能够根据输入和前向传播过程自动构建计算图，并执行反向传播。本节将介绍如何使用autograd包来进行自动求梯度的有关操作。

## 1 概念
上一节介绍的Tensor是这个包的核心类，如果将其属性.requires_grad设置为True，它将开始追踪(track)在其上的所有操作（这样就可以利用链式法则进行梯度传播了）。完成计算后，可以调用.backward()来完成所有梯度计算。此Tensor的梯度将累积到.grad属性中。

注意在y.backward()时，如果y是标量，则不需要为backward()传入任何参数；否则，需要传入一个与y同形的Tensor。解释见 2.3.2 节。

如果不想要被继续追踪，可以调用.detach()将其从追踪记录中分离出来，这样就可以防止将来的计算被追踪，这样梯度就传不过去了。此外，还可以用with torch.no_grad()将不想被追踪的操作代码块包裹起来，这种方法在评估模型的时候很常用，因为在评估模型时，我们并不需要计算可训练参数（requires_grad=True）的梯度。

Function是另外一个很重要的类。Tensor和Function互相结合就可以构建一个记录有整个计算过程的有向无环图（DAG）。每个Tensor都有一个.grad_fn属性，该属性即创建该Tensor的Function, 就是说该Tensor是不是通过某些运算得到的，若是，则grad_fn返回一个与这些运算相关的对象，否则是None。

下面通过一些例子来理解这些概念。
## 2 Tensor

In [1]:
import torch

In [3]:
# 创建一个Tensor，并设置requires_grad=True
x = torch.ones(2,3, requires_grad=True)
x

tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)

In [7]:
print(x.grad_fn)

None


In [8]:
y = x+3
y

tensor([[4., 4., 4.],
        [4., 4., 4.]], grad_fn=<AddBackward0>)

In [9]:
y.grad_fn

- 注意x是直接创建的，所以它没有grad_fn, 而y是通过一个加法操作创建的，所以它有一个为<AddBackward>的grad_fn。

像x这种直接创建的称为叶子节点，叶子节点对应的grad_fn是None。

In [10]:
print(x.is_leaf, y.is_leaf)

True False


- 复杂一些的操作

In [12]:
z = y * y *3
out = z.mean()
print(z)
print(out)

tensor([[48., 48., 48.],
        [48., 48., 48.]], grad_fn=<MulBackward0>)
tensor(48., grad_fn=<MeanBackward0>)


- 通过.requires_grad_()来用inplace的方式改变requires_grad的属性

In [14]:
a = torch.randn(2, 2)  # 默认情况下requires_grad=False
a

tensor([[-1.1037, -0.1229],
        [-0.1833,  2.3274]])

In [15]:
a = ((a*2)/(a-1))
print(a.requires_grad)

False


In [16]:
a.requires_grad_(True)
print(a.requires_grad)

True


In [17]:
b = (a**2).sum()
b.grad_fn

## 梯度
- 因为out是一个标量，所以调用backward()时不需要指定求导变量：

In [18]:
out.backward()  #等价于out.backward(torch.tensor(1.))

In [19]:
x.grad

tensor([[4., 4., 4.],
        [4., 4., 4.]])

- 上面是out关于x的梯度，即$\frac{d_{out}}{d_x}$.
- 令out=O, 因为：
- $$O=\frac{1}{6}\sum^{6}_{i=1}z_i = \frac{1}{6}\sum^{6}_{i=1}3(x_i + 3)^2$$

In [ ]:
3x^2+18x+27;
6x+18